In [1]:
import math

from itertools import combinations

In [2]:
# packages = list(range(1,5+1)) + list(range(7,11+1))
day24_inputs = "inputs/day24.txt"
with open(day24_inputs, 'r') as file:
    day24_data = file.readlines()
packages = [int(x.strip()) for x in day24_data]
print(packages)

[1, 3, 5, 11, 13, 17, 19, 23, 29, 31, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]


In [3]:
def find_combinations(packages, target):
    result = []
    
    def backtrack(start, path, total):
        if total == target:
            result.append(tuple(path))
            return
        if total > target:
            return
        for i in range(start, len(packages)):
            # add current package and update running total
            backtrack(i+1, path + [packages[i]], total + packages[i])
    
    backtrack(0, [], 0)
    return result


In [4]:
def knap_sack_items(target_weight, item_weights, item_values, num_considering):
    if num_considering == 0 or target_weight == 0:
        return 0, []

    # skip item if too heavy 
    if item_weights[num_considering-1] > target_weight:
        return knap_sack_items(target_weight, item_weights, item_values, num_considering-1)

    include_val, include_items = knap_sack_items(target_weight - item_weights[num_considering-1], item_weights, item_values, num_considering-1)
    include_val += item_values[num_considering-1]
    include_items = include_items + [item_weights[num_considering-1]]

    exclude_val, exclude_items = knap_sack_items(target_weight, item_weights, item_values, num_considering-1)

    if include_val > exclude_val:
        return include_val, include_items
    else:
        return exclude_val, exclude_items

In [5]:
def find_package_combinations(num_compartments, packages, max_results=5):
    total_weights = sum(packages)
    target_weight = total_weights // num_compartments
    print(f"for {num_compartments} compartments, each should have a weight of {target_weight}")
    
    all_combinations_packages = find_combinations(packages, target_weight)
    print(f"for a target weight of {target_weight}/compartment there are {len(all_combinations_packages)} potential combinations")
    
    results = []
    best_qe = None
    for combo in all_combinations_packages:
        # take combo from generated combinations, assume this was calculated
        # using the knapsack algorithm, and use the algorithm for the second 
        # and third loops
        
        quantum_entanglement = math.prod(combo)
        # perform early pruning to avoid unnecessary calculations
        if best_qe is not None and quantum_entanglement > best_qe:
            continue

        groups = [list(combo)]
        prev_selected_packages = list(combo)
        
        valid = True
        for i in range(num_compartments-1):
            curr_weight = [x for x in packages if x not in prev_selected_packages]
            if not curr_weight:
                valid = False
                break
            curr_profit = curr_weight         
            max_value, selected_packages = knap_sack_items(target_weight, curr_weight, curr_profit, len(curr_profit))
            
            groups.append(selected_packages)
            prev_selected_packages += selected_packages

        if not valid:
            continue

        total_group_sum = sum([x for xs in groups for x in xs])
        if total_group_sum == total_weights:
            results.append({
                "quantum_entanglement": quantum_entanglement,
                "groups": groups,
            }) 
            results = sorted(results, key=lambda x: x["quantum_entanglement"])[:max_results]
            best_qe = results[0]["quantum_entanglement"]

    print("")
    print(f"there are {len(results)} potential options")
    results_sorted = sorted(results, key=lambda d: d['quantum_entanglement'])
    for option in results_sorted:
        print(f"quantum entanglement of {option["quantum_entanglement"]} and group lens of {[len(group) for group in option["groups"]]}")

In [6]:
find_package_combinations(3, packages, 5)

for 3 compartments, each should have a weight of 516
for a target weight of 516/compartment there are 211830 potential combinations

there are 5 potential options
quantum entanglement of 11266889531 and group lens of [6, 14, 8]
quantum entanglement of 156599427885 and group lens of [8, 12, 8]
quantum entanglement of 15132589773015 and group lens of [10, 10, 8]
quantum entanglement of 2004372679702965 and group lens of [12, 10, 6]
quantum entanglement of 730680502089438615 and group lens of [14, 8, 6]


In [7]:
# part1: given the above printed options, the quantum entanglement (QE) of the
# first group of packages in the ideal configuration is  11266889531 - it has
# the lowest QE and the smallest number of items in the first group 

In [8]:
find_package_combinations(4, packages, 5)

for 4 compartments, each should have a weight of 387
for a target weight of 387/compartment there are 52801 potential combinations

there are 5 potential options
quantum entanglement of 77387711 and group lens of [5, 11, 7, 5]
quantum entanglement of 1057352865 and group lens of [7, 11, 5, 5]
quantum entanglement of 78916435455 and group lens of [9, 9, 5, 5]
quantum entanglement of 14652118228605 and group lens of [11, 7, 5, 5]
quantum entanglement of 5499096450127995 and group lens of [13, 5, 5, 5]


In [9]:
# part2: similar filtering logic as above, 77387711 is our answer to part2